    Normally IPython only displays the output of the last statement. However it can be handy to run multiple sql magics in a single cell and see the output of each execution. Setting `ast_node_interactivity` to `all` will enable that.


In [1]:
# Display all cell outputs in notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
%load_ext ipython_magic.trino

In [3]:

%config Trino.host='localhost'
%config Trino.port=8080
%config Trino.httpScheme='http'
%config Trino.auth=None
%config Trino.user='the-user'

%config Trino.cacheTTL=3600
%config Trino.outputFile="/tmp/trinodb.schema.json"

# comma seperated list of schema to cache in the schema file
%config Trino.catalogs="system,tpch"


    In production environment you will want to pass in an authentiction

```python
    import trino
    %config Trino.auth=trino.auth.BasicAuthentication("principal id", "password")
    %config Trino.user=None
```
    See https://github.com/trinodb/trino-python-client/blob/master/trino/auth.py for more details


In [4]:
%trino --refresh all

Exporting functions: [########################################] 100.0%
Schema file updated: /tmp/trinodb.schema.json


In [5]:
%trino SELECT 'hello'

_col0
hello


In [6]:
#%trino SHOW CATALOGS

In [7]:
%%trino --limit 2 --output sql
SELECT *
FROM
    tpch.tiny.orders AS ord


SELECT *
FROM
    tpch.tiny.orders AS ord

In [8]:
%%trino --limit 2 --dataframe x --output grid
SELECT
    ord.orderkey,
    ord.custkey,
    ord.orderstatus,
    ord.totalprice,
    ord.orderdate,
    ord.orderpriority,
    ord.clerk,
    ord.shippriority,
    ord.comment
FROM
    tpch.tiny.orders AS ord




Saved results to pandas dataframe named `x`
Only showing top 2 row(s)


DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

In [9]:
%%trino --catalog tpch --schema sf1000

SELECT * FROM lineitem

tpch
sf1000
Only showing top 20 row(s)


orderkey,partkey,suppkey,linenumber,quantity,extendedprice,discount,tax,returnflag,linestatus,shipdate,commitdate,receiptdate,shipinstruct,shipmode,comment
3000000001,125756325,8256338,1,19.0,26125.76,0.02,0.0,N,O,1998-05-09,1998-06-08,1998-05-17,TAKE BACK RETURN,REG AIR,asymptotes.
3000000001,186541406,6541407,2,3.0,4314.24,0.08,0.07,N,O,1998-05-22,1998-05-11,1998-05-30,TAKE BACK RETURN,AIR,elets. express pa
3000000001,1410068,3910069,3,32.0,31295.68,0.02,0.02,N,O,1998-05-06,1998-05-10,1998-05-26,NONE,TRUCK,eposits. unusual accounts are
3000000001,99006841,1506851,4,30.0,52286.7,0.01,0.01,N,O,1998-05-31,1998-05-31,1998-06-27,TAKE BACK RETURN,SHIP,ngside of the furiously ironic depo
3000000002,179859525,9859526,1,31.0,45741.43,0.05,0.03,A,F,1992-05-25,1992-07-23,1992-06-24,TAKE BACK RETURN,AIR,lithely carefully even packages. t
3000000003,61893132,9393151,1,48.0,53857.92,0.05,0.0,A,F,1994-04-23,1994-04-23,1994-05-09,NONE,RAIL,sual accounts--
3000000003,37859885,2859892,2,23.0,42388.77,0.02,0.06,R,F,1994-06-14,1994-05-13,1994-06-28,NONE,FOB,deposits. foxes b
3000000003,111086297,1086298,3,4.0,5110.96,0.04,0.05,A,F,1994-05-18,1994-05-10,1994-06-08,NONE,SHIP,. daringly even accounts w
3000000003,27385749,7385750,4,28.0,51334.64,0.06,0.08,A,F,1994-03-19,1994-05-03,1994-04-14,TAKE BACK RETURN,TRUCK,s detect carefully. fluffily enticing requ
3000000003,72277665,4777673,5,35.0,57366.75,0.05,0.04,R,F,1994-04-20,1994-05-18,1994-04-28,NONE,FOB,y unusual


In [10]:
%%trino
SELECT
    lin.orderkey,
    lin.partkey,
    lin.suppkey,
    lin.linenumber,
    lin.quantity,
    lin.extendedprice,
    lin.discount,
    lin.tax,
    lin.returnflag,
    lin.linestatus,
    lin.shipdate,
    lin.commitdate,
    lin.receiptdate,
    lin.shipinstruct,
    lin.shipmode,
    lin.comment,
    ord.orderpriority
FROM
    tpch.sf1000.lineitem AS lin
    INNER JOIN tpch.sf1.orders AS ord ON ord.orderkey = lin.orderkey




Only showing top 20 row(s)


orderkey,partkey,suppkey,linenumber,quantity,extendedprice,discount,tax,returnflag,linestatus,shipdate,commitdate,receiptdate,shipinstruct,shipmode,comment,orderpriority
1,155189345,7689361,1,17.0,24252.03,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,egular courts above the,5-LOW
1,67309080,7309081,2,36.0,39085.92,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,ly final dependencies: slyly bold,5-LOW
1,63699776,3699777,3,8.0,14180.72,0.1,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,"riously. regular, express dep",5-LOW
1,2131495,4631496,4,28.0,42738.92,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,lites. fluffily even de,5-LOW
1,24026634,1526641,5,24.0,37426.32,0.1,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,pending foxes. slyly re,5-LOW
1,15634450,634453,6,32.0,44277.44,0.07,0.02,N,O,1996-01-30,1996-02-07,1996-02-03,DELIVER IN PERSON,MAIL,arefully slyly ex,5-LOW
2,106169722,1169743,1,38.0,67883.96,0.0,0.05,N,O,1997-01-28,1997-01-14,1997-02-02,TAKE BACK RETURN,RAIL,ven requests. deposits breach a,1-URGENT
3,4296962,1796963,1,45.0,88143.75,0.06,0.0,R,F,1994-02-02,1994-01-04,1994-02-23,NONE,AIR,ongside of the furiously brave acco,5-LOW
3,19035429,6535433,2,49.0,66810.03,0.1,0.0,R,F,1993-11-09,1993-12-20,1993-11-24,TAKE BACK RETURN,RAIL,unusual accounts. eve,5-LOW
3,128448229,3448254,3,27.0,31611.6,0.06,0.07,A,F,1994-01-16,1993-11-22,1994-01-23,DELIVER IN PERSON,SHIP,nal foxes wake.,5-LOW


In [11]:
%%trino --output json
SELECT
    1 AS a,
    'abc' as b,
    1.2 as c,
    ARRAY[1,2] as d,
    ARRAY[1, null, 4] as e,
    ARRAY[ARRAY[1,2],ARRAY[5,4]] as f,
    CAST(ROW(1,23,456) as ROW(k1 INT, k2 INT, k3 INT)) as g,
    CAST(ROW(1,'abc',true,null) as ROW(k1 INT, k2 VARCHAR, k3 BOOLEAN, k4 VARCHAR)) as h
    


<IPython.core.display.JSON object>

In [12]:
%%trino?

Docstring:
::

  %trino [-l max_rows] [-r all|local|none] [-d name]
             [-o sql|json|html|grid|text|skip|none] [-s] [-x] [-c catalogname]
             [-m schemaname] [-j]
             [sql [sql ...]]

Magic that works both as %trino and as %%trino

positional arguments:
  sql                   SQL statement to execute

optional arguments:
  -l max_rows, --limit max_rows
                        The maximum number of rows to display. A value of zero
                        is equivalent to `--output skip`
  -r <all|local|none>, --refresh <all|local|none>
                        Force the regeneration of the schema cache file. The
                        `local` option will only update tables/views created
                        in the local Spark context.
  -d name, --dataframe name
                        Capture results in pandas dataframe
  -o <sql|json|html|grid|text|skip|none>, --output <sql|json|html|grid|text|skip|none>
                        Output format. Defaults to